In [163]:
import pandas as pd 
import re
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from PIL import Image

from load_classes import fetch_classdict

In [192]:
class_dict = fetch_classdict()
class_dict

{'N': 'Normal',
 'D': 'Diabetes',
 'G': 'Glaucoma',
 'C': 'Cataract',
 'A': 'Age',
 'H': 'Hypertension',
 'M': 'Pathological',
 'O': 'Other'}

In [232]:
df = pd.read_csv('metadata.csv').drop('Unnamed: 0', axis=1)
df.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,Normal,Diabetes,Glaucoma,Cataract,Age,Hypertension,Pathological,Other,filepath,filename,Labels,Diagnosis,Count
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../ODIR-data/ODIR-5K/ODIR-5K/Training Images/0...,0_right.jpg,['C'],['Cataract'],1
1,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../ODIR-data/ODIR-5K/ODIR-5K/Training Images/0...,0_left.jpg,['C'],['Cataract'],1
2,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../ODIR-data/ODIR-5K/ODIR-5K/Training Images/1...,1_left.jpg,['N'],['Normal'],1
3,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../ODIR-data/ODIR-5K/ODIR-5K/Training Images/1...,1_right.jpg,['N'],['Normal'],1
4,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../ODIR-data/ODIR-5K/ODIR-5K/Training Images/2...,2_right.jpg,"['D', 'O']","['Diabetes', 'Other']",2


### To Do 
- [x] ~Programmatically copy the contents of Training and Testing Images into one folder.~
- [x] ~First, Slice the dataframe to get only single record patient data.~
- [x] ~Then Engineer the Diagnosis from the missing fundus image and create a record.~
- [x] ~Iteratively create rows and sort them at the end using the ID.~
- [x] ~Merge this data back with the original Metadf and save progress.~


### Todo
The Unpaired Records can be used as the final testing set to classify their actual diagnosis.
- [x] Create unpaired_df from the from the records which are not paired accordingly.
- [x] Move the filepath in the df and its inv_orientation into a newfolder called Testing
- [ ] Split the remaining data into Training and validation.
- [ ] Remove Unwanted classes like post-analysing.

In [168]:
ndf = df.copy(deep=True)

In [169]:
ndf['ID'].value_counts()[ndf['ID'].value_counts()==1]

89      1
114     1
69      1
2244    1
90      1
       ..
4169    1
470     1
2727    1
2729    1
485     1
Name: ID, Length: 324, dtype: int64

In [170]:
ids = ndf['ID'].value_counts()[ndf['ID'].value_counts()==1].sort_index().index
print(len(indexes))
ids

324


Int64Index([   2,    3,   22,   25,   38,   39,   41,   50,   59,   63,
            ...
            4394, 4427, 4442, 4448, 4460, 4522, 4551, 4580, 4601, 4659],
           dtype='int64', length=324)

In [226]:
df_list = []
for idx in ids:
    df_list.append(ndf[ndf['ID']==idx])
unpaired_df = pd.concat(df_list)

In [228]:
unpaired_df.index

Int64Index([   4,    5,   38,   43,   64,   65,   68,   85,  100,  107,
            ...
            5900, 5959, 5984, 5993, 6012, 6109, 6162, 6215, 6252, 6353],
           dtype='int64', length=324)

In [175]:
unpaired_df = unpaired_df.reset_index(drop=True)

In [176]:
unpaired_df.head(3)

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,Normal,Diabetes,Glaucoma,Cataract,Age,Hypertension,Pathological,Other,filepath,filename,Labels,Diagnosis,Count
0,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../ODIR-data/ODIR-5K/ODIR-5K/Training Images/2...,2_right.jpg,"['D', 'O']","['Diabetes', 'Other']",2
1,3,66,Male,3_left.jpg,3_right.jpg,normal fundus,branch retinal artery occlusion,0,0,0,0,0,0,0,1,../ODIR-data/ODIR-5K/ODIR-5K/Training Images/3...,3_left.jpg,['O'],['Other'],1
2,22,55,Female,22_left.jpg,22_right.jpg,moderate non proliferative retinopathy，laser spot,laser spot，moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../ODIR-data/ODIR-5K/ODIR-5K/Training Images/2...,22_left.jpg,"['D', 'O']","['Diabetes', 'Other']",2


In [177]:
newcols = ['ID', 'Patient Age', 'Patient Sex', 'Left-Fundus', 'Right-Fundus',
       'left', 'right', 'Normal',
       'Diabetes', 'Glaucoma', 'Cataract', 'Age', 'Hypertension',
       'Pathological', 'Other', 'filepath', 'filename', 'Labels',
       'Diagnosis','Count']
unpaired_df.columns = newcols
unpaired_df.head(2)

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,left,right,Normal,Diabetes,Glaucoma,Cataract,Age,Hypertension,Pathological,Other,filepath,filename,Labels,Diagnosis,Count
0,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../ODIR-data/ODIR-5K/ODIR-5K/Training Images/2...,2_right.jpg,"['D', 'O']","['Diabetes', 'Other']",2
1,3,66,Male,3_left.jpg,3_right.jpg,normal fundus,branch retinal artery occlusion,0,0,0,0,0,0,0,1,../ODIR-data/ODIR-5K/ODIR-5K/Training Images/3...,3_left.jpg,['O'],['Other'],1


### Sample Sizes

In [212]:
print('Number of Training Samples: ', {len(os.listdir(os.path.join(path, 'Training Images')))})
print('Number of Testing Samples: ', {len(os.listdir(os.path.join(path, 'Testing Images')))}) 
print('Number of Merged Samples: ', {len(os.listdir(os.path.join(path, 'Merged')))})
print('Number of New Test Samples: ', {len(os.listdir(os.path.join(path, 'Test')))}) 

Number of Training Samples:  {7000}
Number of Testing Samples:  {1000}
Number of Merged Samples:  {8000}
Number of New Test Samples:  {0}


### Moving the Unpaired Id's to New Folder

In [214]:
# os.makedirs('../ODIR-data/ODIR-5K/ODIR-5K/Test')

In [215]:
from tqdm import tqdm

In [216]:
# first fetch all the IDs into a list 

unpaired_ids = unpaired_df['ID'].to_list()

path = '../ODIR-data/ODIR-5K/ODIR-5K/'

# fetch filenames from the filename column and decompose that into segments to get the orientation
for idx in tqdm(upaired_ids):

    filename = unpaired_df[unpaired_df['ID']==idx]['filename'].values[0]
    orientation = filename.split('_')[1].split('.')[0]

    if orientation == 'right':
        inv_orientation = 'left'
    else: 
         inv_orientation = 'right'

    inv_filename = f'{idx}_{inv_orientation}.jpg'
    
    # Move the pair into newfolder
    shutil.move(os.path.join(path,'Merged',filename), os.path.join(path,'Test'))
    shutil.move(os.path.join(path,'Merged',inv_filename), os.path.join(path,'Test'))





100%|██████████| 324/324 [00:00<00:00, 852.00it/s]


In [217]:
print('Number of Merged Samples: ', {len(os.listdir(os.path.join(path, 'Merged')))})
print('Number of New Test Samples: ', {len(os.listdir(os.path.join(path, 'Test')))}) 

Number of Merged Samples:  {7352}
Number of New Test Samples:  {648}


### Splitting Merged into Training and Validation

In [223]:
print(f'New Test samples is {64800/8000 }% of 8000')
print(f'New validation Split size {(8*7352)/100} samples.')

New Test samples is 8.1% of 8000
New validation Split size 588.16 samples.


### Dividing Data into Classes

In [224]:
df

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,Normal,Diabetes,Glaucoma,Cataract,Age,Hypertension,Pathological,Other,filepath,filename,Labels,Diagnosis,Count
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../ODIR-data/ODIR-5K/ODIR-5K/Training Images/0...,0_right.jpg,['C'],['Cataract'],1
1,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../ODIR-data/ODIR-5K/ODIR-5K/Training Images/0...,0_left.jpg,['C'],['Cataract'],1
2,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../ODIR-data/ODIR-5K/ODIR-5K/Training Images/1...,1_left.jpg,['N'],['Normal'],1
3,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../ODIR-data/ODIR-5K/ODIR-5K/Training Images/1...,1_right.jpg,['N'],['Normal'],1
4,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../ODIR-data/ODIR-5K/ODIR-5K/Training Images/2...,2_right.jpg,"['D', 'O']","['Diabetes', 'Other']",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6387,4689,54,Male,4689_left.jpg,4689_right.jpg,mild nonproliferative retinopathy,normal fundus,0,1,0,0,0,0,0,0,../ODIR-data/ODIR-5K/ODIR-5K/Training Images/4...,4689_left.jpg,['D'],['Diabetes'],1
6388,4690,57,Male,4690_left.jpg,4690_right.jpg,mild nonproliferative retinopathy,mild nonproliferative retinopathy,0,1,0,0,0,0,0,0,../ODIR-data/ODIR-5K/ODIR-5K/Training Images/4...,4690_left.jpg,['D'],['Diabetes'],1
6389,4690,57,Male,4690_left.jpg,4690_right.jpg,mild nonproliferative retinopathy,mild nonproliferative retinopathy,0,1,0,0,0,0,0,0,../ODIR-data/ODIR-5K/ODIR-5K/Training Images/4...,4690_right.jpg,['D'],['Diabetes'],1
6390,4784,58,Male,4784_left.jpg,4784_right.jpg,hypertensive retinopathy，age-related macular d...,hypertensive retinopathy，age-related macular d...,0,0,0,0,1,1,0,0,../ODIR-data/ODIR-5K/ODIR-5K/Training Images/4...,4784_right.jpg,"['A', 'H']","['Age', 'Hypertension']",2
